# Module71 Boosting Assignment2

Q1. What is Gradient Boosting Regression?

A1. Gradient Boosting Regression is an ensemble learning method for regression tasks. It builds an additive model in a sequential manner, combining multiple weak learners (e.g., shallow decision trees). Each weak learner corrects the errors made by its predecessors by minimizing a specified loss function (e.g., Mean Squared Error) using gradient descent.

# Key features include:

1.) **Loss Function:** Optimizes a differentiable loss function (e.g., Mean Squared Error for regression).

2.) **Sequential Training:** Each subsequent model learns to correct the residual errors from the previous ones.

3.) **Weights:** Adjusts predictions iteratively to improve accuracy.


Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy.

A2. Here's a Python implementation of Gradient Boosting Regression from scratch:

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Dataset (simple regression problem)
X = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)
y = np.array([1.2, 2.8, 3.6, 4.5, 5.1])

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []
        self.model_weights = []

    def fit(self, X, y):
        # Initialize with the mean of the target
        self.models.append(np.mean(y))
        residuals = y - self.models[0]

        for _ in range(self.n_estimators):
            # Fit a simple decision stump (shallow tree) on residuals
            tree = DecisionStump(max_depth=self.max_depth)
            tree.fit(X, residuals)
            predictions = tree.predict(X)

            # Update residuals
            residuals -= self.learning_rate * predictions

            # Store the tree
            self.models.append(tree)

    def predict(self, X):
        # Start with the initial mean
        predictions = np.full(X.shape[0], self.models[0])
        for tree in self.models[1:]:
            predictions += self.learning_rate * tree.predict(X)
        return predictions

class DecisionStump:
    def __init__(self, max_depth=2):
        self.threshold = None
        self.feature_index = None
        self.left_value = None
        self.right_value = None

    def fit(self, X, residuals):
        best_mse = float("inf")
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = X[:, feature_index] > threshold

                left_residuals = residuals[left_mask]
                right_residuals = residuals[right_mask]

                # Calculate the mean squared error
                mse = (
                    (left_residuals ** 2).sum() + (right_residuals ** 2).sum()
                ) / n_samples

                if mse < best_mse:
                    best_mse = mse
                    self.threshold = threshold
                    self.feature_index = feature_index
                    self.left_value = left_residuals.mean()
                    self.right_value = right_residuals.mean()

    def predict(self, X):
        predictions = np.full(X.shape[0], 0.0)
        left_mask = X[:, self.feature_index] <= self.threshold
        right_mask = X[:, self.feature_index] > self.threshold
        predictions[left_mask] = self.left_value
        predictions[right_mask] = self.right_value
        return predictions


# Train the model
gbr = GradientBoostingRegressor(n_estimators=10, learning_rate=0.1, max_depth=2)
gbr.fit(X, y)

# Predictions
y_pred = gbr.predict(X)

# Evaluate
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 0.6907556678075526
R-squared: 0.6298994493101411


Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth to
optimise the performance of the model. Use grid search or random search to find the best
hyperparameters.

A3. Here’s how you can experiment with different hyperparameters like learning rate, number of trees, and tree depth in Gradient Boosting Regression using Grid Search and Random Search.

We’ll use scikit-learn's GradientBoostingRegressor for simplicity and efficiency.

1.) **Grid Search for Hyperparameter Tuning:**

Grid Search involves searching exhaustively over a predefined set of hyperparameter combinations.


In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Dataset (example regression problem)
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([1.2, 2.8, 3.6, 4.5, 5.1])

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)

# Hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [1, 2, 3]
}

# Grid Search
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X, y)

# Best parameters and performance
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X)

print("Best Parameters:", best_params)
print("Mean Squared Error:", mean_squared_error(y, y_pred))
print("R-squared:", r2_score(y, y_pred))


Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 100}
Mean Squared Error: 2.0490690472299029e-16
R-squared: 0.9999999999999999


2.) **Random Search for Hyperparameter Tuning**

Random Search randomly samples combinations of hyperparameters over a specified range. It is often faster than Grid Search.

In [3]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Hyperparameter distribution
param_distributions = {
    'n_estimators': [50, 100, 200],
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': [1, 2, 3, 4]
}

# Random Search
random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_distributions,
    n_iter=10,  # Number of random combinations to try
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=1
)
random_search.fit(X, y)

# Best parameters and performance
best_params = random_search.best_params_
best_model = random_search.best_estimator_
y_pred = best_model.predict(X)

print("Best Parameters:", best_params)
print("Mean Squared Error:", mean_squared_error(y, y_pred))
print("R-squared:", r2_score(y, y_pred))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'learning_rate': 0.13022300234864176, 'max_depth': 4, 'n_estimators': 200}
Mean Squared Error: 2.01548754367405e-16
R-squared: 0.9999999999999999


3.) **Explanation of Key Hyperparameters:**

a.) **n_estimators:** The number of boosting stages (trees) to use. Higher values can improve accuracy but may lead to overfitting.

b.) **learning_rate:** Controls the contribution of each tree. Lower values require more trees for accurate results but help prevent overfitting.

c.) **max_depth:** Limits the depth of individual trees. Deeper trees capture more complexity but risk overfitting.

4.) **Comparing Results:**

After running both Grid Search and Random Search, compare the results and choose the method that:

Finds the best hyperparameters faster.
Produces a model with lower Mean Squared Error (MSE) and higher R².

Q4. What is a weak learner in Gradient Boosting?

A4. A weak learner is a model that performs slightly better than random guessing (e.g., a shallow decision tree or decision stump).

In Gradient Boosting, multiple weak learners are combined to form a strong predictive model.

Each weak learner focuses on correcting the errors of its predecessors.

Q5. What is the intuition behind the Gradient Boosting algorithm?

A5. Gradient Boosting minimizes the error (loss function) by:

1.) Training a weak learner to predict residual errors from the previous step.

2.) Adding the weak learner's predictions to the ensemble in a way that reduces the overall loss.

3.) Using gradient descent to optimize the model step by step.

The idea is to move predictions closer to the true values by focusing on the hardest-to-predict examples.

Q6. How does the Gradient Boosting algorithm build an ensemble of weak learners?

A6. The Gradient Boosting algorithm build an ensemble of weak learners by following ways :

1.) **Initialization:** Start with a base prediction (e.g., the mean of the target variable).

2.) **Compute Residuals:** Calculate the residuals (errors) between actual values and predictions.

3.) **Train Weak Learner:** Fit a weak learner to predict the residuals.

4.) **Update Predictions:** Add the weak learner's predictions (scaled by a learning rate) to the ensemble.

5.) **Repeat:** Iterate this process to gradually reduce residual errors.


Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting?

A7. The steps involved in constructing the mathematical intuition of Gradient Boosting are -

1.) **Define the Loss Function:**

Choose a differentiable loss function L(y, f(x)) (e.g., Mean Squared Error).

2.) **Compute the Negative Gradient:**

Approximate the residuals as the negative gradient of the loss function:
``` - ∂L / ∂f(xi) ```

3.) **Fit Weak Learner:**

Train a weak learner (e.g., decision tree) to predict the residuals (negative gradients).

4.) **Update Model:**

Add the weak learner's scaled predictions to the ensemble:
``` f(x) <- f(x) + η * h(x) ``` , where η is the learning rate.

5.) **Iterate:**

Repeat steps 2-4 for a fixed number of iterations or until the error stops decreasing.